# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
# Initial Code from the Exercise
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Read the CSV file
jeopardy = pd.read_csv("jeopardy.csv")

# Inspect jeopardy.csv
print(jeopardy.head(5))

   Show Number    Air Date      Round                         Category  Value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY   $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES   $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...   $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE   $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES   $200   

                                                                                                      Question  \
0             For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory   
1  No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves   
2                     The city of Yuma in this state has a record average of 4,055 hours of sunshine each year   
3                         In 1963, live on "The Art Linkl

After looking over the file, each of the column names have a leading space. In addition, there are two columns with more than one word. We will rename the columns to make the DataFrame easier to work with.

In [2]:
# Rename the columns in jeopardy
jeopardy.columns = ["Show_Number", "Air_Date", "Round", "Category", "Value", "Question", "Answer"]

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

To understand how to create the function, we will first understand how to achieve the results via brute-force.

I found help on [realpython.com](https://realpython.com/python-string-contains-substring/#find-a-substring-in-a-pandas-dataframe-column:~:text=Remove%20ads-,Find%20a%20Substring%20in%20a%20pandas%20DataFrame%20Column,-If%20you%20work). This showed that we can find a sub-string in a DataFrame using the .str.contains() function on a pandas column.

In [3]:
# Brute-Force Method

# We will first create a new DataFrame that searches for "King" in jeopardy.Question
king = jeopardy[jeopardy.Question.str.contains("King")] 

# Next, we will use the newly created DataFrame to search for "England"
king_england = king[king.Question.str.contains("England")]

# We will check if this new DataFrame has the desired results from the question
print(len(king_england)) # The answer is 49 as per the question

49


The Brute-Force Method shows that to find a DataFrame that filters on specific words, we will need to 
1. Create a new DataFrame using .str.contains() and pass the first word as an argument to filter for rows that contain that word
2. Use the newly created DataFrame and repeat the process for the next word
3. Repeat until all words have been found

We will create a function that will take a list of words to filter the Question column and find each row that contains all the words in our list

In [4]:
def find_words(list_of_words):
    # This lambda function will search the Question column for each row that has the passed word
    find = lambda x: jeopardy[jeopardy.Question.str.contains(x)]
    
    # We will need to define the first DataFrame from the list_of_words
    df = find(list_of_words[0])
    
    # If there is only one word to filter on, then we will return the newly created DataFrame
    if len(list_of_words) == 1:
        return df
    
    # If there are more than one word to filter on, then we will use the following
    else:
        for word in list_of_words:
            # The first iteration of df2 will result in the same DataFrame as df
            df2 = df[df.Question.str.contains(word)]
            df = df2
        return df

# Check the find_words function
print(len(find_words(["King", "England"]))) # Answer 49

49


3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

We will first attempt to fix our filter by incorporating case-insensitivity. We can do this by using the .lower() function on each character in the list_of_words passed to our function and to each row in jeopardy.Question.

In [5]:
# Fixing the find_words function to not be case-sensitive
def find_words_case_insensitive(list_of_words):
    # Lowercase the words in the list
    for i in range(len(list_of_words)):
        list_of_words[i] = list_of_words[i].lower()
        
    # Create a new DataFrame from jeopardy with a lowercase Question column
    lowercase_jeopardy = jeopardy
    lowercase_jeopardy["Lowercase_Question"] = lowercase_jeopardy.Question.apply(str.lower)
    
    # This lambda function will search the Lowercase_Question column for each row that has the passed word
    find = lambda x: lowercase_jeopardy[lowercase_jeopardy.Lowercase_Question.str.contains(x)]
    
    # We will need to define the first DataFrame from the list_of_words
    df = find(list_of_words[0])
    
    # # If there is only one word to filter on, then we will return the newly created DataFrame
    if len(list_of_words) == 1:
        return df
    
    # If there are more than one word to filter on, then we will use the following
    else:
        for word in list_of_words:
            # The first iteration of df2 will result in the same DataFrame as df
            df2 = df[df.Lowercase_Question.str.contains(word)]
            df = df2
        return df

# Check the find_words function
print(len(find_words_case_insensitive(["King", "England"]))) # Answer 152

152


4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   While most of the values in the `" Value"` column represent a dollar amount as a string, note that some do not &mdash; these values will need to be handled differently!

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [6]:
# What are the unique values of jeopardy.Value
unique_value = jeopardy.Value.unique()
print(len(unique_value))

150


We will convert any "no value" observations to NANs. Also, we will remove the $ from each of the dollar values, and we will remove any comma's. 

In [7]:
# Create an empty list to append with values
money_list = []

for number in jeopardy.Value:
    if number == "no value":
        money_list.append(float("NaN"))
    else:
        # Remove the $-sign
        x = number.replace("$", "")
    
        # Remove any ,'s
        x = x.replace(",", "")
        
        # Convert the number to a float
        x = float(x)
    
        # Append the money_list
        money_list.append(x)

# Create a new column with float values
jeopardy["Modified_Value"] = money_list

Now that we have a column of values as floats, we can now perform aggregate statistics on that column.

In [8]:
#print(jeopardy_value.head(2))
# What is the average value for a Jeopardy question
print("What is the average value for a Jeopardy question?")
avg_value = jeopardy.Modified_Value.mean()
print(" The average value of a Jeopardy question is $" + str(avg_value.round(2)))

# What is the average value of question with the sub-string "King"?
print("What is the average value of question with the sub-string 'King'?")
x = find_words_case_insensitive(["King"])
avg_value = x.Modified_Value.mean()
print(" The average value of a Jeopardy question with the string 'King' is $" + str(round(avg_value, 2)))

What is the average value for a Jeopardy question?
 The average value of a Jeopardy question is $752.6
What is the average value of question with the sub-string 'King'?
 The average value of a Jeopardy question with the string 'King' is $785.67


We will define a function that will take a given list of words to filter on a DataFrame (similar to Question 3), then find the average value for that question.

In [9]:
def find_words_case_insensitive_avg_value(list_of_words):
    # Define the DataFrame that we'll use to find the average value
    df = find_words_case_insensitive(list_of_words)
    avg_value = df.Modified_Value.mean()
    return avg_value

x = find_words_case_insensitive_avg_value(["King"])
print(round(x, 2)) # Answer is the same as above

785.67


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [10]:
#print(jeopardy.head(3))
x = jeopardy.groupby("Answer").Show_Number.count().reset_index()

def find_words_case_insensitive_unique_answers(list_of_words):
    # Define the DataFrame that we'll use to find the unique answers
    df = find_words_case_insensitive(list_of_words)
    
    # Define the DataFrame with the counts of unique answers
    answer = df.groupby("Answer").Show_Number.count().reset_index()
    return answer

x = find_words_case_insensitive_unique_answers(["King"])
henry_viii = x[x.Answer == "Henry VIII"] # Show Number is 55 as expected

6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

We will explore the additional questions.

First, we will investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word "Computer" compared to questions from the 2000s?

We will need to groupby the year. 

The Air_Date column is a series of strings with YEAR-MONTH-DAY (XXXX-XX-XX). In order to group by year, we will need to split the first four values of each of the dates and add them to a year column.

In [11]:
# For context, what are the earliest and latest dates in our data?
print("The earliest aired date in our data is " + str(jeopardy.Air_Date.min()))
print("The latest aired date in our data is " + str(jeopardy.Air_Date.max()))

The earliest aired date in our data is 1984-09-10
The latest aired date in our data is 2012-01-27


In [12]:
# Define an empty list that we can append
years = []

# Define the dates we will extract the years from
dates = jeopardy.Air_Date

# Iterate over dates to create a list of years
for date in dates:
    x = date[:4]
    years.append(x)
    
# Create a new column in jeopardy to get the year    
jeopardy["Year"] = years

Now that we have a column of years, we are able to groupby year!

In [13]:
df = jeopardy.groupby("Year").Round.count().reset_index()
df.columns = ["Year", "Count"]

print("Here are the amount of episodes for any given year.")
print(df)

Here are the amount of episodes for any given year.
    Year  Count
0   1984   1179
1   1985    888
2   1986   1409
3   1987   1275
4   1988   1290
5   1989   2067
6   1990   4337
7   1991   1444
8   1992   1885
9   1993   2132
10  1994   1136
11  1995   1138
12  1996   4891
13  1997  13099
14  1998  13143
15  1999  13540
16  2000  13439
17  2001  12097
18  2002   6859
19  2003   9426
20  2004  13190
21  2005  13560
22  2006  13726
23  2007  13940
24  2008  14036
25  2009  13579
26  2010  13756
27  2011  13376
28  2012   1093


If we want to group by decade, then we will need to create a decade column.

In [14]:
# Define an empty list that we can append
decades = []

# We will convert each year in years to integers
for i in range(len(years)):
    years[i] = int(years[i])
    
# Iterate over years to create a list of decades
for year in years:
    if year >= 2010:
        decades.append("2010s")
    elif year >= 2000:
        decades.append("2000s")
    elif year >= 1990:
        decades.append("1990s")
    elif year >= 1980:
        decades.append("1980s")
    else:
        decades.append("Check the function")
    
# Create a new column in jeopardy to get the decades    
jeopardy["Decade"] = decades

Now that we have a decades column, we will be able to investigate the ways in which questions change over time by filtering by the decade. 

How many questions from the 90s use the word "Computer" compared to questions from the 2000s?

In [15]:
computer = find_words_case_insensitive(["Computer"])
computer_count = computer.groupby("Decade").Show_Number.count().reset_index()
computer_count.columns = ["Decade", "Count"]
print("Here is a table showcasing the number of times the word 'Computer' was used in a Jeopardy question.")
print(computer_count)
print("There is a difference of " + str(268 - 98) + " times that 'Computer' was used in a Jeopardy question from the 1990s to 2000s.")

Here is a table showcasing the number of times the word 'Computer' was used in a Jeopardy question.
  Decade  Count
0  1980s      6
1  1990s     98
2  2000s    268
3  2010s     59
There is a difference of 170 times that 'Computer' was used in a Jeopardy question from the 1990s to 2000s.


## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.